In [15]:
from preprocessing import * 
from sklearn import svm
from sklearn import pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import cohen_kappa_score
from joblib import dump, load

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Extract data samples from raw GDF files

In [16]:
%%capture
# convert_data()


# Read and split the data

In [17]:
i = 7

x_train, y_train = read_file("action_train", f"A0{i}")
x_test, y_test = read_file("action_test", f"A0{i}")


print("final shapes are: ", x_train.shape, y_train.shape)
print("final shapes are: ", x_test.shape, y_test.shape)


final shapes are:  (230, 25, 750) (230,)
final shapes are:  (58, 25, 750) (58,)


# Extract features from test and train data

In [18]:
%%capture
train_coeff = featurize(x_train)
coeff_len = len(train_coeff)

csp = [mne.decoding.CSP(8) for _ in range(coeff_len)]
x_train_f = np.concatenate(tuple(csp[x].fit_transform(train_coeff[x], y_train) for x  in range(coeff_len)),axis=-1)

test_coeff = featurize(x_test)
x_test_f = np.concatenate(tuple(csp[x].transform(test_coeff[x]) for x  in range(coeff_len)),axis=-1)
  

# Train model and test

In [23]:
clf = pipeline.make_pipeline(StandardScaler(), svm.SVC())
clf.fit(x_train_f, y_train)

print("Accuracy is ", sum(clf.predict(x_train_f) == y_train) / len(x_train))
print(len(x_train))

print("test Accuracy is ", sum(clf.predict(x_test_f) == y_test) / len(x_test))
print(len(x_test))

print("kappa score on train is: ", cohen_kappa_score(clf.predict(x_train_f), y_train))
print("kappa score on test is: ", cohen_kappa_score(clf.predict(x_test_f), y_test))


Accuracy is  1.0
230
test Accuracy is  0.9482758620689655
58
kappa score on train is:  1.0
kappa score on test is:  0.9303721488595438


# Save the trained model

In [24]:
dump(clf, "model.joblib")
dump(csp, "csp.joblib")

['csp.joblib']

In [25]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print(classification_report(y_test, clf.predict(x_test_f)))

              precision    recall  f1-score   support

         1.0       0.94      0.94      0.94        17
         2.0       1.00      0.93      0.97        15
         3.0       0.91      1.00      0.95        10
         4.0       0.94      0.94      0.94        16

    accuracy                           0.95        58
   macro avg       0.95      0.95      0.95        58
weighted avg       0.95      0.95      0.95        58



In [21]:
from sklearn.model_selection import cross_val_score

clf = pipeline.make_pipeline(StandardScaler(), svm.SVC())
scores = cross_val_score(clf, x_train_f, y_train, cv=5)

print(scores, scores.mean(), scores.std())

[0.95652174 0.93478261 1.         0.97826087 1.        ] 0.9739130434782609 0.02535196476019694


In [ ]:
%%capture
outputs= []
acc_train_avg = 0
acc_test_avg = 0
kappa_train_avg = 0
kappa_test_avg = 0
for i in range(1, 10):

  x_train, y_train = read_file("action_train", f"A0{i}")
  x_test, y_test = read_file("action_test", f"A0{i}")


  outputs.append(("final shapes are: ", x_train.shape, y_train.shape))
  outputs.append(("final shapes are: ", x_test.shape, y_test.shape))

  train_coeff = featurize(x_train)
  coeff_len = len(train_coeff)

  csp = [mne.decoding.CSP(8) for _ in range(coeff_len)]
  x_train_f = np.concatenate(tuple(csp[x].fit_transform(train_coeff[x], y_train) for x  in range(coeff_len)),axis=-1)

  test_coeff = featurize(x_test)
  x_test_f = np.concatenate(tuple(csp[x].transform(test_coeff[x]) for x  in range(coeff_len)),axis=-1)

  clf = pipeline.make_pipeline(StandardScaler(), svm.SVC())
  clf.fit(x_train_f, y_train)
  outputs.append("*"*30)
  outputs.append(f"subject {i} is here babeeeeeeeeeee")
  outputs.append(("Accuracy is ", sum(clf.predict(x_train_f) == y_train) / len(x_train)))
  acc_train_avg += sum(clf.predict(x_train_f) == y_train) / len(x_train)
  outputs.append(len(x_train))

  outputs.append(("test Accuracy is ", sum(clf.predict(x_test_f) == y_test) / len(x_test)))
  outputs.append(len(x_test))
  acc_test_avg += sum(clf.predict(x_test_f) == y_test) / len(x_test)

  outputs.append(("kappa score on train is: ", cohen_kappa_score(clf.predict(x_train_f), y_train)))
  outputs.append(("kappa score on test is: ", cohen_kappa_score(clf.predict(x_test_f), y_test)))
  kappa_train_avg += cohen_kappa_score(clf.predict(x_train_f), y_train)
  kappa_test_avg += cohen_kappa_score(clf.predict(x_test_f), y_test)


In [2]:
%%capture
outputs= []
acc_train_avg = 0
acc_test_avg = 0
kappa_train_avg = 0
kappa_test_avg = 0

i = 1
x_train, y_train = read_file("idle_train", f"A0{i}")
x_test, y_test = read_file("idle_test", f"A0{i}")
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))
x, y = shuffle(x, y)

outputs.append(("final shapes are: ", x.shape, y.shape))

k = 10
fold_size = x.shape[0] / 10

indices = [i*int(fold_size) for i in range(1, 10)]
foldsx = np.split(x, indices)
foldsy = np.split(y, indices)
for i in range(len(foldsx)):

  x_test = foldsx[i]
  y_test = foldsy[i]
  restx = (foldsx[j] for j in range(len(foldsx)) if j!=i)
  resty = (foldsy[j] for j in range(len(foldsx)) if j!=i)
  x_train = np.concatenate([a for a in restx])
  y_train = np.concatenate([a for a in resty])
  
  train_coeff = featurize(x_train)
  coeff_len = len(train_coeff)

  csp = [mne.decoding.CSP(8) for _ in range(coeff_len)]
  x_train_f = np.concatenate(tuple(csp[x].fit_transform(train_coeff[x], y_train) for x  in range(coeff_len)),axis=-1)

  test_coeff = featurize(x_test)
  x_test_f = np.concatenate(tuple(csp[x].transform(test_coeff[x]) for x  in range(coeff_len)),axis=-1)

  clf = pipeline.make_pipeline(StandardScaler(), svm.SVC())
  clf.fit(x_train_f, y_train)
  outputs.append(("Accuracy is ", sum(clf.predict(x_train_f) == y_train) / len(x_train)))
  acc_train_avg += sum(clf.predict(x_train_f) == y_train) / len(x_train)
  outputs.append(len(x_train))

  outputs.append(("test Accuracy is ", sum(clf.predict(x_test_f) == y_test) / len(x_test)))
  outputs.append(len(x_test))
  acc_test_avg += sum(clf.predict(x_test_f) == y_test) / len(x_test)

  outputs.append(("kappa score on train is: ", cohen_kappa_score(clf.predict(x_train_f), y_train)))
  outputs.append(("kappa score on test is: ", cohen_kappa_score(clf.predict(x_test_f), y_test)))
  kappa_train_avg += cohen_kappa_score(clf.predict(x_train_f), y_train)
  kappa_test_avg += cohen_kappa_score(clf.predict(x_test_f), y_test)




In [4]:
for i in outputs:
  print(i)
print("average accuracy on train is ", acc_train_avg /10)
print("average accuracy on test is ", acc_test_avg /10)
print("average kappa on train is ", kappa_train_avg /10)
print("average kappa on test is ", kappa_test_avg /10)

# print(acc_train_avg / 9)
# print(acc_test_avg / 9)
# print(kappa_train_avg / 9)
# print(kappa_test_avg / 9)

('final shapes are: ', (410, 25, 750), (410,))
('Accuracy is ', 0.986449864498645)
369
('test Accuracy is ', 0.926829268292683)
41
('kappa score on train is: ', 0.9668183371400824)
('kappa score on test is: ', 0.8353413654618473)
('Accuracy is ', 0.989159891598916)
369
('test Accuracy is ', 0.926829268292683)
41
('kappa score on train is: ', 0.9740970832894599)
('kappa score on test is: ', 0.8081123244929798)
('Accuracy is ', 0.9728997289972899)
369
('test Accuracy is ', 0.926829268292683)
41
('kappa score on train is: ', 0.9338567433856744)
('kappa score on test is: ', 0.8093023255813954)
('Accuracy is ', 0.981029810298103)
369
('test Accuracy is ', 0.9512195121951219)
41
('kappa score on train is: ', 0.9532852259779719)
('kappa score on test is: ', 0.8948717948717949)
('Accuracy is ', 0.978319783197832)
369
('test Accuracy is ', 0.975609756097561)
41
('kappa score on train is: ', 0.9470721124538315)
('kappa score on test is: ', 0.9424964936886395)
('Accuracy is ', 0.991869918699187)
